In [10]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re
import os
from dotenv import load_dotenv
load_dotenv()
import cohere
co = cohere.Client(os.getenv("COHERE_API_KEY"))

from rich import print
from rich import pretty
pretty.install()


import weaviate
import pandas as pd
import numpy as np
from icecream import ic
from utils import dense_retrieval, print_result, keyword_search
from rich import print


In [3]:
# auth_config = weaviate.auth.AuthApiKey(
#     api_key=os.environ['WEAVIATE_API_KEY'])
# client = weaviate.Client(
#     url=os.environ['WEAVIATE_API_URL'],
#     auth_client_secret=auth_config,
#     additional_headers={
#         "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
#     }
# )

client = weaviate.Client(
    auth_client_secret = weaviate.AuthApiKey(api_key = os.getenv("WEAVIATE_API_KEY")),

    # 10 million records from 10 different languages. One million records for each language.
    # Each row is a paragraph from wikipedia
    url = "https://cohere-demo.weaviate.network/",
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),

    }
)


In [2]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""


# Chunking

In [3]:
# # Split into a list of sentences
# texts = text.split('.')

# # Clean up to remove empty spaces and new lines
# texts = np.array([t.strip(' \n') for t in texts])
# texts


array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
       'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',

In [7]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])
print(texts)


['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.'
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical and miniature

In [8]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])
print(texts)


['Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.'
 'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstella

# Now lets Embed the paragraph

In [11]:
response = co.embed(texts = texts.tolist()).embeddings
embeds = np.array(response)
embeds.shape


default model on embed will be deprecated in the future, please specify a model in the request.


(3, 4096)

# Create search index with annoy

In [13]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')


True

In [14]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])

  return results


In [15]:
query = "How much did the film make?"
search(query)


default model on embed will be deprecated in the future, please specify a model in the request.


['Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles.\nIn the United States, it was first released on film stock, expanding to venues using digital projectors.\nThe film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.\nIt received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.\nIt has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.\nInterstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades'
 'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins 

,texts,distance
0,"Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles.\nIn the United States, it was first released on film stock, expanding to venues using digital projectors.\nThe film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.\nIt received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.\nIt has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.\nInterstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades",1.092562
1,"Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.",1.150132
2,"Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.",1.203064


# Reranking

In [4]:
query = "What is the capital of Canada?"
dense_retrieval_results = dense_retrieval(query, client)


In [9]:
print_result(dense_retrieval_results)


item 0
_additional:{'distance': -150.8031}

lang:en

text:The governor general of the province had designated Kingston as the capital in 1841. However, the major population centres of Toronto and Montreal, as well as the former capital of Lower Canada, Quebec City, all had legislators dissatisfied with Kingston. Anglophone merchants in Quebec were the main group supportive of the Kingston arrangement. In 1842, a vote rejected Kingston as the capital, and study of potential candidates included the then-named Bytown, but that option proved less popular than Toronto or Montreal. In 1843, a report of the Executive Council recommended Montreal as the capital as a more fortifiable location and commercial centre, however, the Governor General refused to execute a move without a parliamentary vote. In 1844, the Queen's acceptance of a parliamentary vote moved the capital to Montreal.

title:Ottawa

url:https://en.wikipedia.org/wiki?curid=22219

views:2000


item 1
_additional:{'distance': -150

## Improving Keyword Search with Rerank

In [ ]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=500
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    # print(result.get('text'))


In [13]:
def rerank_responses(query, responses, num_responses=10):
    reranked_responses = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = responses,
        top_n = num_responses,
        )
    return reranked_responses


In [14]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_1, texts)

for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()


i:0

RerankResult<document['text']: Selection of Ottawa as the capital of Canada predates the Confederation of Canada. 
The selection was contentious and not straightforward, with the parliament of the United Province of Canada holding
more than 200 votes over several decades to attempt to settle on a legislative solution to the location of the 
capital., index: 407, relevance_score: 0.9875684>

i:1

RerankResult<document['text']: Montreal was the capital of the Province of Canada from 1844 to 1849, but lost its 
status when a Tory mob burnt down the Parliament building to protest the passage of the Rebellion Losses Bill. 
Thereafter, the capital rotated between Quebec City and Toronto until in 1857, Queen Victoria herself established 
Ottawa as the capital due to strategic reasons. The reasons were twofold. First, because it was located more in the
interior of the Province of Canada, it was less susceptible to attack from the United States. Second, and perhaps 
more importantly, because it lay on the border between French and English Canada, Ottawa was seen as a compromise 
between Montreal, Toronto, Kingston and Quebec City, which were all vying to become the young nation's official 
capital. Ottawa retained the status as capital of Canada when the Province of Canada joined with Nova Scotia and 
New Brunswick to form the Dominion of Canada in 1867., index: 100, relevance_score: 0.9795897>

i:2

...

RerankResult<document['text']: Ottawa is headquarters to numerous major medical organizations and institutions such
as Canadian Red Cross, Canadian Blood Services, Health Canada, Canadian Medical Association, Royal College of 
Physicians and Surgeons of Canada, Canadian Nurses Association, and the Medical Council of Canada., index: 394, 
relevance_score: 0.86153823>

i:8

RerankResult<document['text']: Ontario ( ; ) is one of the thirteen provinces and territories of Canada. Located in
Central Canada, it is Canada's most populous province, with 38.3 percent of the country's population, and is the 
second-largest province by total area (after Quebec). Ontario is Canada's fourth-largest jurisdiction in total area
when the territories of the Northwest Territories and Nunavut are included. It is home to the nation's capital 
city, Ottawa, and the nation's most populous city, Toronto, which is Ontario's provincial capital., index: 228, 
relevance_score: 0.4989891>

i:9

RerankResult<document['text']: With sixty percent of Canada's steel produced in Hamilton by Stelco and Dofasco, the
city has become known as the Steel Capital of Canada. After nearly declaring bankruptcy, Stelco returned to 
profitability in 2004. On August 26, 2007 United States Steel Corporation acquired Stelco for C$38.50 in cash per 
share, owning more than 76 percent of Stelco's outstanding shares. On September 17, 2014, US Steel Canada announced
it was applying for bankruptcy protection and it would close its Hamilton operations., index: 5, relevance_score: 
0.49455282>

## Improving Dense Search with Rerank

In [15]:
query_2 = "Who is the tallest person in history?"
results = dense_retrieval(query_2,client)

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))
    print()


i:0

Robert Wadlow

Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, 
was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and 
raised in Alton, Illinois, a small city near St. Louis, Missouri.

i:1

Manute Bol

...

Sultan Kösen

Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days early by visiting the Ripley's
Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized statue of Robert Wadlow, the 
tallest man ever at 272 cm (8 ft 11.1 in).

i:4

Netherlands

The Dutch are the tallest people in the world, by nationality, with an average height of for adult males and for 
adult females in 2009. The average height of young males in the Netherlands increased from 5 feet, 4 inches to 
approximately 6 feet between the 1850s until the early 2000s. People in the south are on average about shorter than
those in the north.

In [16]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_2, texts)
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()


i:0

RerankResult<document['text']: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton 
Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is 
irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri., index: 0, 
relevance_score: 0.9734939>

i:1

RerankResult<document['text']: Sultan Kösen (born 10 December 1982) is a Turkish farmer who holds the Guinness 
World Record for tallest living male at . Of Kurdish ethnicity, he is the seventh tallest man in history., index: 
2, relevance_score: 0.8664718>

i:2

RerankResult<document['text']: The Dutch are the tallest people in the world, by nationality, with an average 
height of for adult males and for adult females in 2009. The average height of young males in the Netherlands 
increased from 5 feet, 4 inches to approximately 6 feet between the 1850s until the early 2000s. People in the 
south are on average about shorter than those in the north., index: 4, relevance_score: 0.80162543>

i:3

RerankResult<document['text']: Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days
early by visiting the Ripley's Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized 
statue of Robert Wadlow, the tallest man ever at 272 cm (8 ft 11.1 in)., index: 3, relevance_score: 0.6874202>

i:4

RerankResult<document['text']: Bol came from a family of extraordinarily tall men and women. He said: "My mother 
was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and 
the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, 
Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a
village, where you cannot measure yourself," Bol reflected. "I learned I was 7 foot 7 in 1979, when I was grown. I 
was about 18 or 19.", index: 1, relevance_score: 0.6396235>